# Gathering Data

In [13]:
from functions import *
from scivol import *
import nibabel as nib
import ipyvolume as ipv
import numpy as np
import json
import nilearn
from nilearn import masking

In [14]:
proj_root = parent_directory()
print(proj_root)
mri_input_filepath = os.path.join(proj_root, "media/sub-01/anat/sub-01_T2w.nii.gz")
output_folder = os.path.join(proj_root, "output/")
mask_seq_path = "/Users/joachimpfefferkorn/repos/neurovolume/output/slice_images_mask"

/Users/joachimpfefferkorn/repos/neurovolume


# Skull Stripping
Using [nilearn](https://nilearn.github.io/stable/index.html)

In [21]:
anat_img = nib.load(mri_input_filepath)
brain_img = nilearn.masking.compute_brain_mask(anat_img)
full_anat = create_volume(normalize_array(anat_img.get_fdata()))
brain_anat = create_volume(normalize_array(brain_img.get_fdata()))

In [22]:
ipv.quickvolshow(brain_anat)

/Users/joachimpfefferkorn/repos/neurovolume/.venv/lib/python3.11/site-packages/ipyvolume/serialize.py:102: RuntimeWarning: invalid value encountered in cast
  subdata[..., i] = ((gradient[i][zindex] / 2.0 + 0.5) * 255).astype(np.uint8)


Container(children=[VBox(children=(HBox(children=(Label(value='levels:'), FloatSlider(value=0.1, max=1.0, step…

In [23]:
ipv.quickvolshow(full_anat)

Container(children=[VBox(children=(HBox(children=(Label(value='levels:'), FloatSlider(value=0.1, max=1.0, step…

# Building Scivol

In [24]:
affine = anat_img.affine
print(type(affine))

<class 'numpy.ndarray'>


Initialize the scivol object

In [25]:
skullstrip = Scivol("skullstrip", anat_img.affine, 0.2)

Create and add the grids

In [30]:
skullstrip.add_grids([Grid("full_anat",[full_anat]),
Grid("brain_anat", [brain_anat])])

writing full_anat to grid
writing brain_anat to grid


In [36]:
skullstrip.save_scivol(output_folder)

saving skullstrip.scivol
writing skullstrip.scivol


Before moving to blender to render these images, let's make sure that we can parse these scivol files here

In [37]:
scivol_path = "/Users/joachimpfefferkorn/repos/neurovolume/output/skullstrip.scivol"
with open(scivol_path, 'r') as file:
    data = json.load(file)
print("scivol name: ", data['name'], "\n")
for grid_key in data['grids'].keys():
    print(grid_key)
    vol = np.array(data['grids'][grid_key]['frames'][0])

scivol name:  skullstrip 

full_anat
brain_anat


In [38]:
print(data['tolerance'])
print((np.asarray(data['affine'])))

0.2
[[  -0.5           0.           -0.          129.70300293]
 [  -0.            0.5          -0.         -110.68299866]
 [   0.            0.            4.          -74.83930206]
 [   0.            0.            0.            1.        ]]


In [39]:
print(data['grids'].keys())
for grid in data['grids']:
    print(grid)

dict_keys(['full_anat', 'brain_anat'])
full_anat
brain_anat


Cool, let's quickly visualize this

In [40]:
ipv.quickvolshow(np.array(data['grids']['full_anat']['frames'][0]))
# only 0 as it's just 3D, no temporal dimension involved here

Container(children=[VBox(children=(HBox(children=(Label(value='levels:'), FloatSlider(value=0.1, max=1.0, step…

Great! Now lets go build and use a blender plugin